In [12]:
import os
import sys
from datetime import datetime

import copy
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import VOCDetection
import torchvision.transforms as transforms
from torchvision.models import alexnet, resnet50
from torchvision.transforms import functional as F
from xml.etree import ElementTree as ET
import pandas as pd
from PIL import Image

# models from torchvision
import torchvision.models as models
from torchvision.ops import RoIPool

sys.path.append('../../')  
from Object_Detection.RCNN import selective_search
from torchmetrics import Accuracy
from torchinfo import summary

from torch.utils.tensorboard import SummaryWriter


# Get Fast RCNN from torchvision


In [2]:
def parse_annotations(annotation):
    objects = annotation['annotation']['object']
    boxes = []
    classes = []
    for obj in objects:
        xmin = float(obj['bndbox']['xmin'])
        ymin = float(obj['bndbox']['ymin'])
        xmax = float(obj['bndbox']['xmax'])
        ymax = float(obj['bndbox']['ymax'])
        boxes.append([xmin, ymin, xmax, ymax])
        classes.append(obj['name'])
    return torch.tensor(boxes), classes  # Convert boxes to tensors, keep classes as list or map them to integers


In [3]:
# Define the transformation
transform = transforms.Compose([
    transforms.Resize((800, 800)),  # Resize images to a common size
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class CustomVOCDataset(VOCDetection):
    def __getitem__(self, index):
        img, target = super(CustomVOCDataset, self).__getitem__(index)
        boxes, classes = parse_annotations(target)
        return img, boxes, classes

# Update the dataset instance with the custom class
dataset = CustomVOCDataset(root='./data/VOCdevkit/VOC2012', year='2012', image_set='train', download=True, transform=transform)
data_loader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=0)


Using downloaded and verified file: ./data/VOCdevkit/VOC2012\VOCtrainval_11-May-2012.tar
Extracting ./data/VOCdevkit/VOC2012\VOCtrainval_11-May-2012.tar to ./data/VOCdevkit/VOC2012


In [4]:
img, boxes, classes = dataset[0]

In [8]:
base_model = models.resnet50(weights=True)

# Remove the final fully connected layer (fc) to use as a feature extractor
modules = list(base_model.children())[:-2]  # Remove the last fully connected layer and avgpool
base_model = torch.nn.Sequential(*modules)


In [ ]:
# Intermediate RoI pooling layer that takes selected region proposals and extracts features
# RoI pooling
roi_pool = RoIPool(output_size=(7, 7), spatial_scale=1.0)


In [15]:
num_classes = 21  # PASCAL VOC classes + background
classifier = torch.nn.Sequential(
    torch.nn.Linear(512 * 7 * 7, 4096),
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(4096, 4096),
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(4096, num_classes + 4)  # class scores and bbox regressor outputs
)

